<a href="https://colab.research.google.com/github/dishadtu/Pair_Trading_Copulas/blob/main/Summary_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

singular = pd.read_csv('/content/Strategy Result Mixture_with_both_volatilities.csv')
mixture=pd.read_csv('/content/Strategy Result_For_Selected_Singular.csv')

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print("Number of rows in singular:", len(singular))
print("Number of rows in mixture:", len(mixture))


Number of rows in singular: 16
Number of rows in mixture: 16


In [ ]:
print(singular.columns)
print(mixture.columns)

Index(['stock1', 'stock2', 'daily_volatility', 'annual_volatility',
       'sharpe_ratio', 'cagr'],
      dtype='object')
Index(['stock1', 'stock2', 'daily_volatility', 'annual_volatility',
       'sharpe_ratio', 'cagr'],
      dtype='object')


In [ ]:
print(mixture.columns)

Index(['stock1', 'stock2', 'daily_volatility', 'annual_volatility',
       'sharpe_ratio', 'cagr'],
      dtype='object')


In [ ]:
# Ensure numeric types for calculations
singular['sharpe_ratio'] = pd.to_numeric(singular['sharpe_ratio'])
singular['cagr'] = pd.to_numeric(singular['cagr'])
singular['annual_volatility'] = pd.to_numeric(singular['annual_volatility'])
singular['daily_volatility'] = pd.to_numeric(singular['daily_volatility'])

In [ ]:
mixture['sharpe_ratio'] = pd.to_numeric(mixture['sharpe_ratio'])
mixture['cagr'] = pd.to_numeric(mixture['cagr'])
mixture['annual_volatility'] = pd.to_numeric(mixture['annual_volatility'])
mixture['daily_volatility'] = pd.to_numeric(mixture['daily_volatility'])

In [ ]:
# Calculate differences (mixture - singular)
sharpe_diff = mixture['sharpe_ratio'] - singular['sharpe_ratio']
cagr_diff = mixture['cagr'] - singular['cagr']
annual_vol_diff = mixture['annual_volatility'] - singular['annual_volatility']
daily_vol_diff = mixture['daily_volatility'] - singular['daily_volatility']

In [ ]:
# Create a difference table
diff_table = pd.DataFrame({
    'stock1': singular['stock1'],
    'stock2': singular['stock2'],
    'sharpe_diff': sharpe_diff,
    'cagr_diff': cagr_diff,
    'annual_vol_diff': annual_vol_diff,
    'daily_vol_diff': daily_vol_diff
})

In [ ]:
# Calculate summary statistics
summary_stats = {
    'Metric': ['Sharpe Ratio', 'CAGR', 'Annual Volatility', 'Daily Volatility'],
    'Singular Mean': [
        singular['sharpe_ratio'].mean(),
        singular['cagr'].mean(),
        singular['annual_volatility'].mean(),
        singular['daily_volatility'].mean()
    ],
    'Singular Std': [
        singular['sharpe_ratio'].std(),
        singular['cagr'].std(),
        singular['annual_volatility'].std(),
        singular['daily_volatility'].std()
    ],
    'Mixture Mean': [
        mixture['sharpe_ratio'].mean(),
        mixture['cagr'].mean(),
        mixture['annual_volatility'].mean(),
        mixture['daily_volatility'].mean()
    ],
    'Mixture Std': [
        mixture['sharpe_ratio'].std(),
        mixture['cagr'].std(),
        mixture['annual_volatility'].std(),
        mixture['daily_volatility'].std()
    ],
    'Mean Improvement': [
        sharpe_diff.mean(),
        cagr_diff.mean(),
        annual_vol_diff.mean(),
        daily_vol_diff.mean()
    ],
    'Std Improvement': [
        sharpe_diff.std(),
        cagr_diff.std(),
        annual_vol_diff.std(),
        daily_vol_diff.std()
    ],
    'Percent Improvement': [
        (sharpe_diff.mean() / singular['sharpe_ratio'].mean()) * 100 if singular['sharpe_ratio'].mean() != 0 else np.nan,
        (cagr_diff.mean() / singular['cagr'].mean()) * 100 if singular['cagr'].mean() != 0 else np.nan,
        (annual_vol_diff.mean() / singular['annual_volatility'].mean()) * 100,
        (daily_vol_diff.mean() / singular['daily_volatility'].mean()) * 100
    ]
}

# Create summary DataFrame
summary_df = pd.DataFrame(summary_stats)

In [ ]:
# Perform paired t-tests
sharpe_ttest = ttest_rel(mixture['sharpe_ratio'], singular['sharpe_ratio'])
cagr_ttest = ttest_rel(mixture['cagr'], singular['cagr'])
annual_vol_ttest = ttest_rel(mixture['annual_volatility'], singular['annual_volatility'])
daily_vol_ttest = ttest_rel(mixture['daily_volatility'], singular['daily_volatility'])

# Add p-values to summary DataFrame
summary_df['p-value'] = [
    sharpe_ttest.pvalue,
    cagr_ttest.pvalue,
    annual_vol_ttest.pvalue,
    daily_vol_ttest.pvalue
]

In [ ]:
# Add significance indicator
alpha = 0.05
summary_df['Significant'] = summary_df['p-value'] < alpha

# Count improvements
sharpe_improved = (sharpe_diff > 0).sum()
cagr_improved = (cagr_diff > 0).sum()
total_pairs = len(singular)

print(f"Summary Statistics:\n{summary_df}\n")
print(f"Sharpe Ratio improved in {sharpe_improved}/{total_pairs} pairs ({sharpe_improved/total_pairs*100:.1f}%)")
print(f"CAGR improved in {cagr_improved}/{total_pairs} pairs ({cagr_improved/total_pairs*100:.1f}%)")
print("\nDetailed t-test results:")
print(f"Sharpe Ratio: t={sharpe_ttest.statistic:.4f}, p={sharpe_ttest.pvalue:.4f}")
print(f"CAGR: t={cagr_ttest.statistic:.4f}, p={cagr_ttest.pvalue:.4f}")
print(f"Annual Volatility: t={annual_vol_ttest.statistic:.4f}, p={annual_vol_ttest.pvalue:.4f}")



Summary Statistics:
              Metric  Singular Mean  Singular Std  Mixture Mean  Mixture Std  \
0       Sharpe Ratio      -0.026778      0.508298     -0.567142     0.808046   
1               CAGR       0.564458      1.686375      0.068601     0.288188   
2  Annual Volatility       1.472621      1.066246      0.251831     0.361132   
3   Daily Volatility       0.092766      0.067167      0.015864     0.022749   

   Mean Improvement  Std Improvement  Percent Improvement   p-value  \
0         -0.540363         0.604685          2017.912843  0.002767   
1         -0.495857         1.405530           -87.846579  0.178602   
2         -1.220791         0.985038           -82.899163  0.000172   
3         -0.076903         0.062052           -82.899163  0.000172   

   Significant  
0         True  
1        False  
2         True  
3         True  

Sharpe Ratio improved in 2/16 pairs (12.5%)
CAGR improved in 7/16 pairs (43.8%)

Detailed t-test results:
Sharpe Ratio: t=-3.5745, p=0.00

In [ ]:
# Save the difference table for further analysis
diff_table.to_csv('performance_differences.csv', index=False)
print("\nDetailed differences table saved to 'performance_differences.csv'")



Detailed differences table saved to 'performance_differences.csv'


In [ ]:
# Save summary statistics to a CSV file
summary_df.to_csv('summary_statistics.csv', index=False)